Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

**SET COVERING**

We have a set to cover with some *subsets*, we cover the set when taking subsets that *complete* the set even if some tiles are taken multiple times.

In [231]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [232]:
PROBLEM_SIZE = 20
NUM_SETS = 40
SETS = tuple(
    np.array([random() < 0.25 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [233]:
SETS

(array([False,  True,  True, False, False, False,  True, False, False,
        False,  True,  True,  True, False, False, False,  True, False,
         True, False]),
 array([False, False, False,  True,  True, False, False,  True, False,
        False,  True, False, False,  True,  True, False,  True, False,
         True, False]),
 array([False, False,  True, False, False,  True, False,  True,  True,
        False, False,  True, False, False, False,  True, False, False,
         True, False]),
 array([False, False,  True, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False,  True,
        False, False]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False,  True, False,
        False, False]),
 arr

We reach the goal if taken sets cover all the set of "tiles" which number is PROBLEM_SIZE.

In [234]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"


We try to solve it with a Breadth-first search, just to have numbers to compare. For a Breadth-First search we use a SimpleQueue().

In [235]:
def breadth_first_search(state): 
    frontier = SimpleQueue()
    frontier.put(state)

    counter = 0
    current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put(new_state)
        current_state = frontier.get()
    
    print(f"Solved in {counter:,} steps ({len(current_state.taken)} set of tiles)")
    print(f"Solution for Breadth First Search: {current_state}")


Now, we want to implement $A^*$, so we define actual cost $g(\cdot)$ and the heuristic $h(\cdot)$. Each state $n$ will be inserted in the priority queue depending on the value $f(n)=g(n)+h(n)$. 

In [236]:
def actual_cost(state):
    #actual cost (g) can be seen as the number of state taken
    return len(state.taken)

The distance is a pessimistic function, so it can be a candidate for heuristic.

In [237]:
def distance(state):
    #the heuristic can be seen as the number of the remaining tiles to occupy, indeed we can think about in a pessimistic way: 
    #each tile could be given at least from one set, so we would need n set to cover n tiles!
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

The first heuristic I tried to implement was about:
- we have NUM_SETS and PROBLEM_SIZE
- since every set is generated with a probability < 0.3, we can say that we would need tiles * 0.3 at least to cover (rounded up), but in general not_taken * 0.3

In [238]:
def heuristic_1(state):
    return int(len(state.not_taken)/(len(state.not_taken)*0.3)) + 1 


In [239]:
def a_star_search(state, h = distance, g=True):
    frontier = PriorityQueue()
    if g:
        frontier.put((actual_cost(state) + h(state), state))
    else:
        frontier.put((h(state), state))
    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        #print(f'Current state is: {current_state}')
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            if g:
                frontier.put((actual_cost(new_state) + h(new_state), new_state))
            else:
                frontier.put((h(new_state), new_state))
            #print(f'Frontier: {frontier.queue}')
        _, current_state = frontier.get()

    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} set of tiles)"
    )

    print(f"Solution for A* Search: {current_state}")


Now we solve it with both searches so we can see the difference in terms of performance:

In [240]:
#generate the state 
state = State(set(), set(range(NUM_SETS)))

In [241]:
breadth_first_search(state)

Solved in 61,414 steps (4 set of tiles)
Solution for Breadth First Search: State(taken={16, 0, 18, 1}, not_taken={2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


In [242]:
a_star_search(state, g=False)

Solved in 4 steps (4 set of tiles)
Solution for A* Search: State(taken={16, 0, 18, 1}, not_taken={2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


In [243]:
a_star_search(state)

Solved in 4 steps (4 set of tiles)
Solution for A* Search: State(taken={16, 0, 18, 1}, not_taken={2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


In [244]:
a_star_search(state, h=heuristic_1)

Solved in 61,181 steps (4 set of tiles)
Solution for A* Search: State(taken={1, 18, 26, 7}, not_taken={0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


With this heuristic we have less step then Breadth-First.

Now we implement an heuristic that returns the number of tiles in a set, so the more tiles the better!

In [245]:
def heuristic_2(state):
    return -np.sum(reduce(
        np.logical_or, 
        [SETS[i] for i in state.not_taken],
        np.zeros(PROBLEM_SIZE)
    ))

def heuristic_3(state):
    h = -np.sum(reduce(
        np.logical_or, 
        [SETS[i] for i in state.taken],
        np.zeros(PROBLEM_SIZE)
    ))
    #print(h)
    return h

In [246]:
a_star_search(state, h=heuristic_2)

Solved in 61,318 steps (4 set of tiles)
Solution for A* Search: State(taken={1, 18, 26, 7}, not_taken={0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


In [247]:
a_star_search(state, h=heuristic_3)

Solved in 4 steps (4 set of tiles)
Solution for A* Search: State(taken={16, 0, 18, 1}, not_taken={2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


another version uses the number of uncovered elements from the subsets, so we can measure "how" they could contribute, but not "how well" unluckily (the quality depends also on the future!).

In [255]:
def heuristic_4(state):
    taken = reduce(
        np.logical_or, 
        [SETS[i] for i in state.taken],
        np.zeros(PROBLEM_SIZE, dtype=np.int8)
    )
    ntlist = []
    for i in state.not_taken:
        x = reduce(
            np.logical_xor,
            [np.array(SETS[i], dtype=np.int8), taken]
        )
        y = x ^ taken
        #print('x - taken', np.sum(y))
        ntlist.append(np.sum(x))
    ntlist.sort(reverse=True)
    #print(f'# uncovered from every not taken set: {ntlist}')
    #now check uncovered tiles - sum (max tiles from set) 
    d = distance(state)
    
    count = 0
    while (d>0) and ntlist:
        d -= ntlist.pop()
        count += 1

    return count

In [256]:
a_star_search(state, h=heuristic_4)

Solved in 216 steps (4 set of tiles)
Solution for A* Search: State(taken={0, 16, 18, 1}, not_taken={2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


it seems to work!
following this trend, we try to mix this and the first heuristic: we take the distance and divide for the largest number of uncovered tiles and "true" tiles. 

In [250]:
def heuristic_5(state):
    taken = reduce(
        np.logical_or, 
        [SETS[i] for i in state.taken],
        np.zeros(PROBLEM_SIZE, dtype=np.int8)
    )
    ntlist = []
    for i in state.not_taken:
        x = reduce(
            np.logical_xor,
            [np.array(SETS[i], dtype=np.int8), taken]
        )
        y = x ^ taken
        ntlist.append(np.sum(y))

    maxN = max(ntlist)
    #print('max', maxN)
    return int(distance(state)/maxN) + 1

def heuristic_6(state):
    #not checking the remaining tiles for n, but just the remaining sets
    sets = (SETS[i] for i in state.not_taken)
    d = distance(state)
    
    maxN = max(np.sum(subset) for subset in sets)
    #print('max', maxN)
    return int(d/maxN) 

In [251]:
#from professor Squillero repo, just for reference and comparison
def h2(state):
    already_covered = reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = distance(state)
    optimistic_estimate = int(missing_size/largest_set_size) + 1
    return optimistic_estimate

In [252]:
a_star_search(state, h=heuristic_5)

Solved in 17,443 steps (4 set of tiles)
Solution for A* Search: State(taken={1, 26, 18, 7}, not_taken={0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


In [253]:
a_star_search(state, h=heuristic_6)

Solved in 17,443 steps (4 set of tiles)
Solution for A* Search: State(taken={1, 26, 18, 7}, not_taken={0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})


In [254]:
a_star_search(state, h=h2)

Solved in 882 steps (4 set of tiles)
Solution for A* Search: State(taken={0, 9, 29, 30}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39})
